# **XML**

Аббревиатура **XML** расшифровывается как eXtensible Markup Language — расширяемый язык разметки. Он (язык) позволяет описывать документы, используя теги.

![пример](https://lms.skillfactory.ru/assets/courseware/v1/f92e35f47fa17ac0ddd40efde0e3389d/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/xml_0.jpg)

In [2]:
import pandas as pd
from IPython.display import display
# Импортируем модуль ElementTree
import xml.etree.ElementTree as ET

##### **ИЗВЛЕКАЕМ КОНТЕНТ ИЗ XML-ФАЙЛА**
***
Данные в формате XML имеют **древовидную** структуру.  
Самый верхнеуровневый узел называется **корнем**, а всё, что находится в самом низу, называется **листьями**.  
Кроме того, у menu есть **дети (потомки)** — это два узла dish, имеющие разное значение атрибута name.  
Для работы с XML-файлами мы будем использовать модуль **ElementTree**, входящий в стандартный пакет xml. Этот модуль позволит нам «перемещаться» по дереву XML и смотреть, что находится в каждом его узле, начиная с корня и заканчивая листьями.  


In [4]:
# считаем содержимое файла в переменную tree
tree = ET.parse('data/menu.xml')

##### **КОРЕНЬ**
***
Запишем в переменную root корневой узел дерева tree и посмотрим, как выглядит содержимое переменной root, для чего выполним код:

In [5]:
root = tree.getroot()
display(root)

<Element 'menu' at 0x000001F27739DA90>

In [6]:
display(type(root))

xml.etree.ElementTree.Element

Если мы вызовем встроенный в Python метод type() и передадим ему root , то увидим, что это тип **xml.etree.ElementTree.Element**. Такой тип будет **у любого узла в дереве**.

##### **ПОТОМКИ**
***
Для того чтобы посмотреть список потомков корневого узла, выполним следующий код (Если у узла нет потомков, то вернётся пустой список — [].):

In [7]:
display(list(root))

[<Element 'dish' at 0x000001F2785ED9A0>,
 <Element 'dish' at 0x000001F2785EDAE0>]

Для того чтобы получить список потомков второго (?)блюда в нашем меню и вывести его на экран, выполним код:

In [8]:
display(list(root[1]))

[<Element 'price' at 0x000001F2785EDB30>,
 <Element 'weight' at 0x000001F2785EDB80>,
 <Element 'class' at 0x000001F2785EDA90>]

##### **АТРИБУТЫ И ТЕГИ**
***
Как было сказано ранее, у узлов могут быть параметры, или атрибуты. Например, у узлов dish есть атрибут name, который хранит название блюда.

In [9]:
# Выведем на экран атрибуты первого блюда из меню:
display(root[0].attrib)

{'name': 'Кура'}

In [10]:
# возьмём узел price первого блюда из меню
display(root[0][0])
# Теперь прочитаем значение этого узла с помощью text
display(root[0][0].text)

<Element 'price' at 0x000001F2785ED8B0>

'40'

Все значения в XML, даже числовые, хранятся как строки, поэтому преобразовывать их к нужному типу придётся самим.  
Например, в данном случае можно обернуть значение стоимости в int() или float().

In [12]:
# получим наименование тега третьего потомка указанного родителя
display(root[0][2].tag)

'class'

##### **ИСПОЛЬЗОВАНИЕ ЦИКЛОВ**
***
Используя цикл for, автоматизируем обход дерева. Для этого напишем следующий код:  

In [13]:
# перебираем всех потомков первого поколения
for dish in root: 
    for param in dish: # перебираем все параметры каждого из потомков
        print(dish.attrib['name'], param.tag, param.text) # выводим: имя потомка, тег параметра, строковое значение параметра
    print() # отделаем вывод каждого потомка пустой строкой

Кура price 40
Кура weight 300
Кура class Мясо

Греча price 20
Греча weight 200
Греча class Крупа



# **XML. Загружаем, создаем, сохраняем**

##### **ЗАГРУЖАЕМ ДАННЫЕ ИЗ XML-ФАЙЛА В DATAFRAME**
***
Cтандартных средств для превращения XML-файла в DataFrame нет, но если XML-файл содержит чёткую структуру, сделать это средствами Python достаточно просто.

In [21]:
column_names = ['name', 'price', 'weight', 'class'] # список названий столбцов
df_list = []

for dish in root: # проходим по каждому потомку
    row = [dish.attrib['name'], dish[0].text, dish[1].text, dish[2].text] # формируем строку будущей таблицы в виде: имя потомка, значение первого узла, второго и т д
    df_list.append(row) # заливаем получившийся список в список
df = pd.DataFrame(df_list, columns=column_names) # просто вставляем список со списками в датафрейм
display(df)

,name,price,weight,class
0,Кура,40,300,Мясо
1,Греча,20,200,Крупа


##### **СОЗДАЁМ XML-ФАЙЛ**
***
Воссоздадим структуру нашего исходного XML-файла с нуля,  руководствуясь общими рекомендациями

In [24]:
# Чтобы создать корень дерева, используем метод Element() из класса ElementTree:
new_root = ET.Element('menu')
display(new_root)
# Теперь мы можем добавлять новые узлы в наше дерево, используя метод SubElement() из того же класса.
# Добавим в наше меню двух потомков корневого узла,
# которые будут представлять два блюда, то есть будут узлами dish:
dish1 = ET.SubElement(new_root, 'dish', name='Кура')
dish2 = ET.SubElement(new_root, 'dish', name='Греча')
display(list(new_root))
# Добавим в создаваемую структуру по три потомка (атрибута) к двум новым узлам,
# которые будут содержать информацию о блюде — о его цене (price),
# весе (weight) и классе (class), а также значение этих атрибутов:
price1 = ET.SubElement(dish1, "price").text = "40"
weight1 = ET.SubElement(dish1, "weight").text = "300"
class1 = ET.SubElement(dish1, "class").text = "Мясо"
display(list(dish1))

price2 = ET.SubElement(dish2, "price").text = "20"
weight2 = ET.SubElement(dish2, "weight").text = "200"
class2 = ET.SubElement(dish2, "class").text = "Крупа"
display(list(dish2))

<Element 'menu' at 0x000001F2785FB590>

[<Element 'dish' at 0x000001F2785EA9A0>,
 <Element 'dish' at 0x000001F2785EAF40>]

[<Element 'price' at 0x000001F2785F5040>,
 <Element 'weight' at 0x000001F2785F5090>,
 <Element 'class' at 0x000001F2786314F0>]

[<Element 'price' at 0x000001F2786315E0>,
 <Element 'weight' at 0x000001F278631680>,
 <Element 'class' at 0x000001F278631590>]

In [25]:
# проверяем
for dish in new_root:
    for param in dish:
        print(dish.attrib['name'], param.tag, param.text)
    print()

Кура price 40
Кура weight 300
Кура class Мясо

Греча price 20
Греча weight 200
Греча class Крупа



##### **СОХРАНЕНИЕ XML-ФАЙЛА**
***
Запишем созданную нами структуру как XML-файл на диск

In [27]:
new_root_string = ET.tostring(new_root)
with open("data/new_menu.xml", "wb") as f:
    f.write(new_root_string)

In [29]:
# запись в нужной кодировке
ET.ElementTree(new_root).write('data/new_menu_good.xml', encoding="utf-8")

Для этого мы передаём в класс ElementTree() наше дерево (не его строковое представление) и вызываем метод write(). В метод мы передаём путь к новому файлу и нужную нам кодировку.

# **ДОП. МАТЕРИАЛЫ**

**ZIP**

* [Working with zip files in Python (англ.)](https://www.geeksforgeeks.org/working-zip-files-python/)

**EXCEL**

* [Автоматизация Excel с помощью Python (рус.)](https://medium.com/nastia-shu/больше-не-нужно-открывать-сотни-файлов-в-excel-e0a1f5a9e9a7)
* [Использование Python и Excel для обработки и анализа данных (рус.)](https://habr.com/ru/company/otus/blog/331998/)
* [How to Work with Excel files in Pandas (англ.)](https://towardsdatascience.com/how-to-work-with-excel-files-in-pandas-c584abb67bfb)
* [Pandas read_excel() – Reading Excel File in Python (англ.)](https://www.journaldev.com/33306/pandas-read_excel-reading-excel-file-in-python)
* [Python Excel Tutorial: The Definitive Guide (англ.)](https://www.datacamp.com/community/tutorials/python-excel-tutorial)
* [Tutorial Using Excel with Python and Pandas (англ.)](https://www.dataquest.io/blog/excel-and-pandas/)

**JSON**

* [Парсинг JSON (рус.)](https://all-python.ru/osnovy/json.html)
* [Working With JSON Data in Python (англ.)](https://realpython.com/python-json/)
* [Python JSON (англ.)](https://www.programiz.com/python-programming/json)

**XML**

* [Работа с XML из Python (рус.)](https://codecamp.ru/blog/python-manipulating-xml/)
* [Processing XML in Python — ElementTree (англ.)](https://towardsdatascience.com/processing-xml-in-python-elementtree-c8992941efd2)